Dependencies

In [1]:
import subprocess
import json
import re
import os
import cma
import csv
import pandas as pd

Datasets

In [2]:
datasets = {
    "xor": {
        "train": "./dat/train_xor.dat",
        "test": "./dat/test_xor.dat"
    },
    "compas": {
        "train": "./dat/train_compas.dat",
        "test": "./dat/test_compas.dat"
    },
    "nomnist": {
        "train": "./dat/train_nomnist.dat",
        "test": "./dat/test_nomnist.dat"
    }
}

Hiperparameters values

In [3]:
# Valores discretos para las capas y las neuronas
if True:    
    layers_range = [1, 2, 3, 4]
    neurons_options = [2, 4, 8, 16, 32, 64, 128]
    offline_options = [0, 1]
    error_function_options = [0, 1]

    # Rango continuo para eta y mu
    eta_range = (0.0001, 0.7)
    mu_range = (0.0001, 1.0)
else:
    layers_range = [1]
    neurons_options = [2, 4]
    offline_options = [0, 1]
    error_function_options = [0, 1]

    # Rango continuo para eta y mu
    eta_range = (0.01, 0.1)
    mu_range = (0.01, 0.1)

In [4]:
#train_test_regex = r"Train error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)\nTest error \(Mean \+- SD\):\s+([\d\.eE+-]+) \+- ([\d\.eE+-]+)"
#train_test_regex = r"Train error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)\s*Test\s+error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)"
#train_test_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)"
train_test_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)"

In [5]:
def run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function):
    # Construir el comando con las nuevas flags
    command = [
        "./bin/la2", "-t", train_file, "-T", test_file,
        "-e", str(eta), "-m", str(mu), "-l", str(layers), "-h", str(neurons), "-i", "2000",
        "-f", str(error_function)
    ]
    # Añadir el modo offline si corresponde
    if offline == 1:
        command.append("-o")

    # Ejecutar el programa
    result = subprocess.run(command, capture_output=True, text=True)

    # Buscar los valores de error
    match = re.search(train_test_regex, result.stdout, re.DOTALL)
    if match:
        train_mean, train_std, test_mean, test_std = match.groups()
        return {
            "train_mean": float(train_mean),
            "train_std": float(train_std),
            "test_mean": float(test_mean),
            "test_std": float(test_std)
        }
    return None

In [6]:
def evaluate_fitness(params, train_file, test_file):
    # Limitar eta y mu dentro de sus rangos permitidos
    eta = max(min(params[0], eta_range[1]), eta_range[0])
    mu = max(min(params[1], mu_range[1]), mu_range[0])

    # Limitar otros parámetros como antes
    layers = int(max(min(params[2], max(layers_range)), min(layers_range)))
    neurons_index = int(max(min(params[3], len(neurons_options) - 1), 0))
    neurons = neurons_options[neurons_index]
    offline = int(max(min(params[4], 1), 0))
    error_function = int(max(min(params[5], 1), 0))

    # Ejecutar el programa y obtener resultados
    result = run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function)

    if result:
        return result["test_mean"], result  # Devuelve el test_mean y el resto del resultado
    return float("inf"), None  # Penalizar si hay fallos y no hay resultado


In [7]:
def save_to_csv(filename, results):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Encabezado del CSV
        writer.writerow(["Eta", "Mu", "Layers", "Neurons", "Offline", "Error Function",
                         "Train Mean", "Train Std", "Test Mean", "Test Std"])
        # Escribir cada fila de resultados
        for row in results:
            writer.writerow(row)

In [8]:
# Bucle principal actualizado
def optimize_with_cma_es(dataset_key):
    if dataset_key not in datasets:
        print(f"Dataset '{dataset_key}' no encontrado.")
        return

    # Archivos de entrenamiento y prueba
    train_file = datasets[dataset_key]["train"]
    test_file = datasets[dataset_key]["test"]

    # Configuración inicial de CMA-ES
    initial_params = [0.01, 0.5, 2, 3, 1, 0]  # [eta, mu, layers, neurons_index, offline, error_function]

    sigma = 0.1 # Rango de búsqueda inicial
    es = cma.CMAEvolutionStrategy(
    initial_params,
    sigma,
    {
        'bounds': [
            [eta_range[0], mu_range[0], 1, 0, 0, 0],  # Límite inferior
            [eta_range[1], mu_range[1], max(layers_range), len(neurons_options) - 1, 1, 1]  # Límite superior
        ]
    }
)

    # Resultados almacenados
    results = []

    # Bucle de optimización
    while not es.stop():
        solutions = es.ask()
        fitness = []
        for solution in solutions:
            # Evaluar cada solución
            test_mean, result = evaluate_fitness(solution, train_file, test_file)
            fitness.append(test_mean)  # Usar solo test_mean como fitness

            if result:  # Guardar resultados solo si son válidos
                eta, mu = solution[0], solution[1]
                layers = int(max(min(solution[2], max(layers_range)), min(layers_range)))
                neurons_index = int(max(min(solution[3], len(neurons_options) - 1), 0))
                neurons = neurons_options[neurons_index]
                offline = int(max(min(solution[4], 1), 0))
                error_function = int(max(min(solution[5], 1), 0))

                # Almacenar resultados
                results.append([
                    eta, mu, layers, neurons, offline, error_function,
                    result["train_mean"], result["train_std"], result["test_mean"], result["test_std"]
                ])

        # Informar a CMA-ES
        es.tell(solutions, fitness)
        print(f"Generación {es.result.iterations}: Mejor error de prueba = {min(fitness)}")


    # Guardar resultados en CSV
    csv_filename = f"results_{dataset_key}.csv"
    save_to_csv(csv_filename, results)
    print(f"Resultados guardados en {csv_filename}")

In [9]:
resultados = []

In [10]:
nombre = "xor"
optimize_with_cma_es(nombre)
resultados.append(pd.read_csv(f"results_{nombre}.csv"))


(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=999069, Sun Dec 22 20:42:19 2024)
Generación 1: Mejor error de prueba = 0.00012217
Generación 2: Mejor error de prueba = 9.9249e-05
Generación 3: Mejor error de prueba = 0.000100223
Generación 4: Mejor error de prueba = 7.66904e-05
Generación 5: Mejor error de prueba = 6.56835e-05
Generación 6: Mejor error de prueba = 7.08655e-05
Generación 7: Mejor error de prueba = 7.79664e-05
Generación 8: Mejor error de prueba = 7.22491e-05
Generación 9: Mejor error de prueba = 6.25398e-05
Generación 10: Mejor error de prueba = 6.12768e-05
Generación 11: Mejor error de prueba = 6.09224e-05
Generación 12: Mejor error de prueba = 6.12289e-05
Generación 13: Mejor error de prueba = 6.23809e-05
Generación 14: Mejor error de prueba = 6.05429e-05
Generación 15: Mejor error de prueba = 6.07448e-05
Generación 16: Mejor error de prueba = 6.03083e-05
Generación 17: Mejor error de prueba = 6.06412e-05
Generación 18: Mejor error de prueba = 6.03532e-05
Gen

In [14]:
print(resultados[0].sort_values(by=["Test Mean", "Train Mean", "Layers", "Neurons"], ascending=True).head(10))

         Eta        Mu  Layers  Neurons  Offline  Error Function  Train Mean  \
15  0.031915  0.720080       2        8        0               0    0.100412   
4   0.071625  0.421034       1        8        0               0    0.000640   
13  0.149025  0.409308       1        8        0               0    0.000554   
0   0.098662  0.486013       1        8        0               0    0.000524   
14  0.123281  0.533195       1        8        0               0    0.000458   
10  0.221538  0.527181       1        8        0               0    0.000394   
18  0.063005  0.729858       1        8        0               0    0.000373   
9   0.137716  0.674372       1        8        0               0    0.000361   
16  0.199086  0.735562       1        8        0               0    0.000311   
7   0.063588  0.521528       1       16        0               0    0.000306   

    Train Std  Test Mean  Test Std  
15   0.200530   0.100412  0.200530  
4    0.000118   0.000640  0.000118  
13   0.0